# **Projet d'etude**

## **1 - Import of libraries and file preparation**

In [ ]:
#Run this cell to install all needed libraries
%pip install tensorflow
%pip install numpy
%pip install matplotlib
%pip install seaborn
%pip install memory_profiler

In [ ]:
from CustomReLU import *
from Models import *
from Test_functions import *
from Config import *
import os


os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

BATCH_SIZE = 40
MNIST = tf.keras.datasets.mnist
CIFAR10 = tf.keras.datasets.cifar10
MLP_PATH_MNIST = 'MNIST/MLP'
CNN_PATH_MNIST = 'MNIST/CNN'
CNN_PATH_CIFAR10 = 'CIFAR10/CNN'

## **2 - Folders and datasets setup**

### 2.1 - Folders setup

Files are stored in CIFAR10 and MNIST folders.

In [ ]:
for dataset_name in ["MNIST", "CIFAR10"]:
    if not os.path.exists(dataset_name):
        os.mkdir(dataset_name)

for _path in [MLP_PATH_MNIST, CNN_PATH_CIFAR10, CNN_PATH_MNIST]:
    if not os.path.exists(_path):
        os.mkdir(_path)
    if not os.path.exists(_path + "/images"):
        os.mkdir(_path + "/images")
    if not os.path.exists(_path + "/models"):
        os.mkdir(_path + "/models")
    for elem in ["/varying_gamma", "/varying_beta", "/varying_clipping", "/varying_beta_gamma", "/varying_beta_clipping", "/varying_gamma_clipping"]:
        if not os.path.exists(_path + "/images" + elem):
            os.mkdir(_path + "/images" + elem)

### 2.2 - Dataset setup

In [ ]:
# MNIST
(X_Train_MNIST, Y_Train_MNIST), (X_Test_MNIST, Y_Test_MNIST) = MNIST.load_data()
X_Train_MNIST, X_Test_MNIST = X_Train_MNIST.astype(
    'float32') / 255.0, X_Test_MNIST.astype('float32') / 255.0
X_Train_MNIST, X_Test_MNIST = np.expand_dims(
    X_Train_MNIST, axis=3), np.expand_dims(X_Test_MNIST, axis=3)
Y_Train_MNIST, Y_Test_MNIST = tf.keras.utils.to_categorical(
    Y_Train_MNIST, num_classes=10), tf.keras.utils.to_categorical(Y_Test_MNIST, num_classes=10)
Train_DS_MNIST = tf.data.Dataset.from_tensor_slices(
    (X_Train_MNIST, Y_Train_MNIST)).shuffle(10000).batch(BATCH_SIZE)
Test_DS_MNIST = tf.data.Dataset.from_tensor_slices(
    (X_Test_MNIST, Y_Test_MNIST)).batch(BATCH_SIZE)

# CIFAR10
(X_Train_CIFAR, Y_Train_CIFAR), (X_Test_CIFAR, Y_Test_CIFAR) = CIFAR10.load_data()
X_Train_CIFAR, X_Test_CIFAR = X_Train_CIFAR.astype(
    'float32') / 255.0, X_Test_CIFAR.astype('float32') / 255.0
Y_Train_CIFAR, Y_Test_CIFAR = tf.keras.utils.to_categorical(
    Y_Train_CIFAR, num_classes=10), tf.keras.utils.to_categorical(Y_Test_CIFAR, num_classes=10)
Train_DS_CIFAR = tf.data.Dataset.from_tensor_slices(
    (X_Train_CIFAR, Y_Train_CIFAR)).shuffle(10000).batch(BATCH_SIZE)
Test_DS_CIFAR = tf.data.Dataset.from_tensor_slices(
    (X_Test_CIFAR, Y_Test_CIFAR)).batch(BATCH_SIZE)

## **3 - Models setup**

### 3.1 - Models configuration

We create a configuration dictionary for a model to have the initialization information of the model such as its input_shape, the list of layers, the dataset used, the number of epochs and the folder to save the image and the weights in .h5 format.

All dictionaries are stored in ```Model_config_list```.

In [ ]:
Model_config_list = []

MNIST_model_mlp_config = {
    "name": "MNIST_model_MLP",
    "dataset": 'MNIST',
    "layer_list": [Flatten(), Dense(32, name='layer0'), ReLU(), Dense(16, name='layer1'), ReLU(), Dense(16, name='layer2'), ReLU(), Dense(10, name='layer3')],
    "input_shape": (None, 28, 28, 1),
    "epochs": 3,
    "path": MLP_PATH_MNIST}
Model_config_list.append(MNIST_model_mlp_config)

MNIST_model_cnn_config = {
    "name": "MNIST_model_CNN",
    "dataset": 'MNIST',
    "layer_list": [Conv2D(32, kernel_size=(3, 3), padding="same", name='layer0'), ReLU(), MaxPooling2D(), Conv2D(32, kernel_size=(3, 3), padding="same", name='layer1'), ReLU(), MaxPooling2D(), Flatten(), Dense(10, name='layer2')],
    "input_shape": (None, 28, 28, 1),
    "epochs": 3,
    "path": CNN_PATH_MNIST}
Model_config_list.append(MNIST_model_cnn_config)

CIFAR10_model_cnn_config = {
    "name": "CIFAR10_model_CNN",
    "dataset": 'CIFAR10',
    "layer_list": [Conv2D(32, kernel_size=(3, 3), padding="same", name='layer0'), ReLU(), MaxPooling2D(), Conv2D(64, kernel_size=(3, 3), padding="same", name='layer1'), ReLU(), MaxPooling2D(), Conv2D(128, kernel_size=(3, 3), padding="same", name='layer2'), ReLU(), MaxPooling2D(), Flatten(), Dense(512, name='layer3'), Dense(10, name='layer4')],
    "input_shape": (None, 32, 32, 3),
    "epochs": 50,
    "path": CNN_PATH_CIFAR10}
Model_config_list.append(CIFAR10_model_cnn_config)

### 3.2 - Models training

Using configuration dictionnaries, models are trained or loaded from file.  

All trained models are stored in ```Trained_model_list```.

In [ ]:
Trained_model_list = []

for model_config in Model_config_list:
    training_model = Training_Model(name=model_config["name"], layers_list=model_config["layer_list"],
                                    input_shape=model_config["input_shape"], path=model_config["path"], dataset_name=model_config["dataset"])
    print("Training model : {}".format(training_model.name))
    training_model.compile(
        loss=tf.keras.losses.CategoricalCrossentropy(),
        optimizer=tf.keras.optimizers.RMSprop(),
        metrics=["accuracy"],
    )
    if os.path.isfile(model_config["path"] + "/models/model_0.h5"):
        training_model.build(input_shape=model_config["input_shape"])
        training_model.load_weights(
            model_config["path"] + "/models/model_0.h5")
        print("\t Model weights loaded from file")
    else:
        training_model.build(input_shape=model_config["input_shape"])
        if model_config['dataset'] == 'MNIST':
            training_model.fit(Train_DS_MNIST, epochs=model_config["epochs"])
        else:
            training_model.fit(Train_DS_CIFAR, epochs=model_config["epochs"])
        training_model.save_weights(
            model_config["path"] + "/models/model_0.h5")
    Trained_model_list.append(training_model)

In [ ]:
del X_Train_MNIST, X_Train_CIFAR, Y_Train_MNIST, Y_Train_CIFAR
del Train_DS_MNIST, Train_DS_CIFAR

### 3.3 - Testing models creation

Once trained, models are transformed using ```Testing_Model``` class to convert ReLUs into CustomReLUs.

All testing models are stored in ```Testing_list```.

In [ ]:
Testing_list = []

for model in Trained_model_list:
    for _config in Config_model_list:
        if _config["name"] == model.name:
            config = _config
    testing_model = Testing_Model(config=config, training_model=model)
    Testing_list.append(testing_model)

## **4 - Model testing varying one parameter**

The models are tested by varying one of the parameters according to the min, max and step variables defined in `Config.py`.

### 4.1 - Beta testing

In [ ]:
for model in Testing_list:
    print("Testing parameter with model : {}".format(model.name))
    if model.dataset_name == 'MNIST':
        fig = plot_varying_parameter_accuracy(model, "beta", model.config["beta_config"]["min"], model.config[
                                              "beta_config"]["max"], model.config["beta_config"]["step"], dataset=Test_DS_MNIST, plot_loss=True)
    else:
        fig = plot_varying_parameter_accuracy(model, "beta", model.config["beta_config"]["min"], model.config[
                                              "beta_config"]["max"], model.config["beta_config"]["step"], dataset=Test_DS_CIFAR, plot_loss=True)
    fig.savefig(model.path + "/images/varying_beta/test_{}.jpg".format(model.name),
                dpi=400, bbox_inches='tight')
    fig.clf()
    print("\n")

### 4.2 - Gamma testing

In [ ]:
for model in Testing_list:
    print("Testing parameter with model : {}".format(model.name))
    if model.dataset_name == 'MNIST':
        fig = plot_varying_parameter_accuracy(
            model, "gamma", model.config["gamma_config"]["min"], model.config["gamma_config"]["max"], model.config["gamma_config"]["step"], dataset=Test_DS_MNIST)
    else:
        fig = plot_varying_parameter_accuracy(
            model, "gamma", model.config["gamma_config"]["min"], model.config["gamma_config"]["max"], model.config["gamma_config"]["step"], dataset=Test_DS_CIFAR)
    fig.savefig(model.path + "/images/varying_gamma/test_{}.jpg".format(model.name),
                dpi=400, bbox_inches='tight')
    fig.clf()
    print("\n")

### 4.3 - Clipping testing

In [ ]:
for model in Testing_list:
    print("Testing parameter with model : {}".format(model.name))
    if model.dataset_name == 'MNIST':
        fig = plot_varying_parameter_accuracy(
            model, "clipping", model.config["clipping_config"]["min"], model.config["clipping_config"]["max"], model.config["clipping_config"]["step"], dataset=Test_DS_MNIST)
    else:
        fig = plot_varying_parameter_accuracy(
            model, "clipping", model.config["clipping_config"]["min"], model.config["clipping_config"]["max"], model.config["clipping_config"]["step"], dataset=Test_DS_CIFAR)
    fig.savefig(model.path + "/images/varying_clipping/test_{}.jpg".format(model.name),
                dpi=400, bbox_inches='tight')
    fig.clf()
    print("\n")

### 4.4 - Clipping percentage testing

In [ ]:
for model in Testing_list:
    print("Testing parameter with model : {}".format(model.name))
    if model.dataset_name == 'MNIST':
        fig = plot_varying_clipping_percentage_accuracy(model, model.config["clipping_percentage_config"]["min"], model.config[
                                                        "clipping_percentage_config"]["max"], model.config["clipping_percentage_config"]["step"], dataset=Test_DS_MNIST, inputs=X_Test_MNIST)
    else:
        fig = plot_varying_clipping_percentage_accuracy(model, model.config["clipping_percentage_config"]["min"], model.config[
                                                        "clipping_percentage_config"]["max"], model.config["clipping_percentage_config"]["step"], dataset=Test_DS_CIFAR, inputs=X_Test_CIFAR)
    fig.savefig(model.path + "/images/varying_clipping/test_percentage_{}.jpg".format(
        model.name), dpi=400, bbox_inches='tight')
    fig.clf()
    print("\n")

## **5 - Model testing varying two parameters** 

The models are tested by varying two of the parameters according to the min, max and step variables defined in `Config.py`.

### 5.1 - Beta - gamma testing

In [ ]:
for model in Testing_list:
    print("Testing parameters with model : {}".format(model.name))
    parameters_list_x = ['beta', model.config["beta_gamma_config"]["beta_min"],
                         model.config["beta_gamma_config"]["beta_max"], model.config["beta_gamma_config"]["beta_step"]]
    parameters_list_y = ['gamma', model.config["beta_gamma_config"]["gamma_min"],
                         model.config["beta_gamma_config"]["gamma_max"], model.config["beta_gamma_config"]["gamma_step"]]
    if model.dataset_name == 'MNIST':
        fig = plot_heatmap_varying_parameters_accuracy(
            model, parameters_list_x, parameters_list_y, dataset=Test_DS_MNIST, inputs=X_Test_MNIST)
    else:
        fig = plot_heatmap_varying_parameters_accuracy(
            model, parameters_list_x, parameters_list_y, dataset=Test_DS_CIFAR, inputs=X_Test_CIFAR)
    fig.savefig(model.path + "/images/varying_beta_gamma/test_{}.jpg".format(
        model.name), dpi=400, bbox_inches='tight')
    fig.clf()
    print("\n")

### 5.2 - Gamma - Clipping testing

In [ ]:
for model in Testing_list:
    print("Testing parameters with model : {}".format(model.name))
    parameters_list_x = ['gamma', model.config["gamma_clipping_config"]["gamma_min"],
                         model.config["gamma_clipping_config"]["gamma_max"], model.config["gamma_clipping_config"]["gamma_step"]]
    parameters_list_y = ['clipping', model.config["gamma_clipping_config"]["clipping_min"],
                         model.config["gamma_clipping_config"]["clipping_max"], model.config["gamma_clipping_config"]["clipping_step"]]
    if model.dataset_name == 'MNIST':
        fig = plot_heatmap_varying_parameters_accuracy(
            model, parameters_list_x, parameters_list_y, dataset=Test_DS_MNIST, inputs=X_Test_MNIST)
    else:
        fig = plot_heatmap_varying_parameters_accuracy(
            model, parameters_list_x, parameters_list_y, dataset=Test_DS_CIFAR, inputs=X_Test_CIFAR)
    fig.savefig(model.path + "/images/varying_gamma_clipping/test_{}.jpg".format(
        model.name), dpi=400, bbox_inches='tight')
    fig.clf()
    print("\n")

### 5.3 - Beta - Clipping testing

In [ ]:
for model in Testing_list:
    print("Testing parameters with model : {}".format(model.name))
    parameters_list_x = ['beta', model.config["beta_clipping_config"]["beta_min"],
                         model.config["beta_clipping_config"]["beta_max"], model.config["beta_clipping_config"]["beta_step"]]
    parameters_list_y = ['clipping', model.config["beta_clipping_config"]["clipping_min"],
                         model.config["beta_clipping_config"]["clipping_max"], model.config["beta_clipping_config"]["clipping_step"]]
    if model.dataset_name == 'MNIST':
        fig = plot_heatmap_varying_parameters_accuracy(
            model, parameters_list_x, parameters_list_y, dataset=Test_DS_MNIST, inputs=X_Test_MNIST)
    else:
        fig = plot_heatmap_varying_parameters_accuracy(
            model, parameters_list_x, parameters_list_y, dataset=Test_DS_CIFAR, inputs=X_Test_CIFAR)
    fig.savefig(model.path + "/images/varying_beta_clipping/test_{}.jpg".format(
        model.name), dpi=400, bbox_inches='tight')
    fig.clf()
    print("\n")

### 5.4 - Beta - Clipping percentage testing

In [ ]:
for model in Testing_list:
    print("Testing parameters with model : {}".format(model.name))
    parameters_list_x = ['beta', model.config["beta_clipping_percentage_config"]["beta_min"],
                         model.config["beta_clipping_percentage_config"]["beta_max"], model.config["beta_clipping_percentage_config"]["beta_step"]]
    parameters_list_y = ['clipping_percentage', model.config["beta_clipping_percentage_config"]["clipping_percentage_min"],
                         model.config["beta_clipping_percentage_config"]["clipping_percentage_max"], model.config["beta_clipping_percentage_config"]["clipping_percentage_step"]]
    if model.dataset_name == 'MNIST':
        fig = plot_heatmap_varying_parameters_accuracy(
            model, parameters_list_x, parameters_list_y, dataset=Test_DS_MNIST, inputs=X_Test_MNIST)
    else:
        fig = plot_heatmap_varying_parameters_accuracy(
            model, parameters_list_x, parameters_list_y, dataset=Test_DS_CIFAR, inputs=X_Test_CIFAR)
    fig.savefig(model.path + "/images/varying_beta_clipping/test_percentage_{}.jpg".format(
        model.name), dpi=400, bbox_inches='tight')
    fig.clf()
    print("\n")

### 5.5 - Gamma - Clipping percentage testing

In [ ]:
for model in Testing_list:
    print("Testing parameters with model : {}".format(model.name))
    parameters_list_x = ['gamma', model.config["gamma_clipping_percentage_config"]["gamma_min"],
                         model.config["gamma_clipping_percentage_config"]["gamma_max"], model.config["gamma_clipping_percentage_config"]["gamma_step"]]
    parameters_list_y = ['clipping_percentage', model.config["gamma_clipping_percentage_config"]["clipping_percentage_min"],
                         model.config["gamma_clipping_percentage_config"]["clipping_percentage_max"], model.config["gamma_clipping_percentage_config"]["clipping_percentage_step"]]
    if model.dataset_name == 'MNIST':
        fig = plot_heatmap_varying_parameters_accuracy(
            model, parameters_list_x, parameters_list_y, dataset=Test_DS_MNIST, inputs=X_Test_MNIST)
    else:
        fig = plot_heatmap_varying_parameters_accuracy(
            model, parameters_list_x, parameters_list_y, dataset=Test_DS_CIFAR, inputs=X_Test_CIFAR)
    fig.savefig(model.path + "/images/varying_gamma_clipping/test_percentage_{}.jpg".format(
        model.name), dpi=400, bbox_inches='tight')
    fig.clf()
    print("\n")

## **6 - Computing optimal ranges**

Optimal ranges are computed to get maximum accuracy using normal distribution.

In [ ]:
for model in Testing_list:
    if model.dataset_name == 'MNIST':
        beta_range = compute_range(
            model, 'beta', 0.02, 5, 2, 2, dataset=Test_DS_MNIST)
        gamma_range = compute_range(
            model, 'gamma', 0.02, 5, 2, 2, dataset=Test_DS_MNIST)
        clipping_percentage_range = compute_range(
            model, 'clipping_percentage', 0.02, 5, 1, 1, dataset=Test_DS_MNIST, inputs=X_Test_MNIST)
    else:
        beta_range = compute_range(
            model, 'beta', 0.02, 5, 2, 2, dataset=Test_DS_CIFAR)
        gamma_range = compute_range(
            model, 'gamma', 0.02, 5, 2, 2, dataset=Test_DS_CIFAR)
        clipping_percentage_range = compute_range(
            model, 'clipping_percentage', 0.02, 5, 1, 1, dataset=Test_DS_CIFAR, inputs=X_Test_CIFAR)

    model.config["beta_range"] = beta_range
    model.config["gamma_range"] = gamma_range
    model.config["clipping_percentage_range"] = clipping_percentage_range

## **7 - Testing with all random parameters**

Random models are generated using parameters in computed optimal ranges.

### 7.1 - Testing models with all random parameters every inference

In [ ]:
for model in Testing_list:
    if model.dataset_name == 'MNIST':
        fig = update_model_c_relus_changing_every_inference(model, model.config["beta_range"], model.config["gamma_range"],  model.config[
                                                            "clipping_percentage_range"], inputs=X_Test_MNIST, outputs=Y_Test_MNIST, nb_test=20, dataset=Test_DS_MNIST)
    else:
        fig = update_model_c_relus_changing_every_inference(model, model.config["beta_range"], model.config["gamma_range"],  model.config[
                                                            "clipping_percentage_range"], inputs=X_Test_CIFAR, outputs=Y_Test_CIFAR, nb_test=20, dataset=Test_DS_CIFAR)
    fig.savefig(
        model.path + "/images/test_random_parameters_every_inference_{}.jpg".format(model.name), dpi=400)
    fig.clf()

### 7.2 - Testing varying models with all random parameters

In [ ]:
for model in Testing_list:
    Testing_list_copies = [model]

    for i in range(10):
        model_copy = model.clone()
        if model.dataset_name == 'MNIST':
            beta, gamma, clipping_percentage = update_model_c_relus_with_ranges(
                model_copy, model.config["beta_range"], model.config["gamma_range"], model.config["clipping_percentage_range"], inputs=X_Test_MNIST)
        else:
            beta, gamma, clipping_percentage = update_model_c_relus_with_ranges(
                model_copy, model.config["beta_range"], model.config["gamma_range"], model.config["clipping_percentage_range"], inputs=X_Test_CIFAR)
        print("Testing model accuracy with beta = {}, gamma = {}, clipping percentage = {}".format(
            beta, gamma, clipping_percentage))
        Testing_list_copies.append(model_copy)

    for i in range(model.activation_nb):
        if model.dataset_name == 'MNIST':
            fig = plot_varying_model_HW_dist(
                Testing_list_copies, activation_index=i, diff=True, inputs=X_Test_MNIST)
        else:
            fig = plot_varying_model_HW_dist(
                Testing_list_copies, activation_index=i, diff=True, inputs=X_Test_CIFAR)
        fig.savefig(
            model.path + "/images/test_random_parameters_hist_{}_activation_layer_{}.jpg".format(model.name, i), dpi=400)
        fig.clf()

In [ ]:
test_model_c_relus_with_ranges(Testing_list[2], [0.75, 1.37], [-0.05,0.03], [0,0.8], Test_DS_CIFAR, X_Test_CIFAR)